In [ ]:
import numpy as np
import pandas as pd
import gc
from os import remove

path = ''

file = path+"train_data.csv"
file_te = path+"test_data.csv"
outputhead = 'feat_1hr.v3.'
daycut_val = pd.to_datetime(pd.Series(['2016-06-01','2016-06-24'])).map(lambda x: x.date()) 
daycut_tr = pd.to_datetime(pd.Series(['2016-05-06','2016-05-31'])).map(lambda x: x.date())
daycut_Rtr = pd.to_datetime(pd.Series(['2016-06-05','2016-06-30'])).map(lambda x: x.date())

valdays = 31
lastday = 182
daycut_agg = np.array([0, lastday-valdays]) #M1-M5

divide = 3600. * 1 #for 1 hours

try:
    remove(path+outputhead+'train.h5')
    remove(path+outputhead+'test.h5')
    print('file '+path+outputhead+'train.h5 removed!')
except OSError:
    pass

In [ ]:
## load data
df_train = pd.read_csv(file, usecols = ['custid','platform','currency',
                                        'groupid','timestamp','invest'])#,nrows=1000000)
days = 182

In [ ]:
## add time of day column
mintimestamp = np.min(df_train['timestamp'])
timestamp = df_train['timestamp'] - mintimestamp
divide = 3600. #for 1 hours
df_train['time_hr'] = (np.rint(np.remainder(timestamp, 3600*24)/(divide)))

In [ ]:
## groupby currency, time
grouped2 = df_train[ (df_train['invest']>=2.0) & (df_train['invest'] < 10.0)].\
           groupby(['currency','time_hr'])
grouped10 = df_train[df_train['invest']>=10.0].groupby(['currency','time_hr'])
grouped = df_train.groupby(['currency','time_hr'])
bin_hr_currency = pd.DataFrame({'meanInv.curHr': grouped['invest'].mean().round(decimals=2), 
'rInvGT2.curHr':(grouped2['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT10.curHr':(grouped10['invest'].count()/grouped['invest'].count()).round(decimals=4),
'q75Inv.curHr':grouped['invest'].quantile(0.75).round(decimals=2), 
'q95Inv.curHr':grouped['invest'].quantile(0.95).round(decimals=2), 
'q999Inv.curHr':grouped['invest'].quantile(0.999).round(decimals=2)})

del grouped2, grouped10, grouped
gc.collect()

In [ ]:
## groupby platform, time
grouped2 = df_train[(df_train['invest']>=2.0) & (df_train['invest'] < 10.0)].\
groupby(['platform','time_hr'])
grouped10 = df_train[df_train['invest']>=10.0].groupby(['platform','time_hr'])
grouped = df_train.groupby(['platform','time_hr'])
bin_hr_pltf = pd.DataFrame({'meanInv.pltfHr': grouped['invest'].mean().round(decimals=2), 
'rInvGT2.pltfHr':(grouped2['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT10.pltfHr':(grouped10['invest'].count()/grouped['invest'].count()).round(decimals=4),
'q75Inv.pltfHr':grouped['invest'].quantile(0.75).round(decimals=2), 
'q95Inv.pltfHr':grouped['invest'].quantile(0.95).round(decimals=2), 
'q999Inv.pltfHr':grouped['invest'].quantile(0.999).round(decimals=2)})
del grouped2, grouped10, grouped
gc.collect()

In [ ]:
## groupby groupid, time_hr
grouped2 = df_train[(df_train['invest']>=2.0) & (df_train['invest'] < 10.0)].\
groupby(['groupid', 'time_hr'])
grouped10 = df_train[df_train['invest']>=10.0].groupby(['groupid','time_hr'])
grouped = df_train.groupby(['groupid', 'time_hr'])
bin_hr_grpid = pd.DataFrame({'meanInv.grpidHr': grouped['invest'].mean().round(decimals=2), 
'rInvGT2.grpidHr':(grouped2['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT10.grpidHr':(grouped10['invest'].count()/grouped['invest'].count()).round(decimals=4),
'q75Inv.grpidHr':grouped['invest'].quantile(0.75).round(decimals=2), 
'q95Inv.grpidHr':grouped['invest'].quantile(0.95).round(decimals=2), 
'q999Inv.grpidHr':grouped['invest'].quantile(0.999).round(decimals=2)})

del grouped2, grouped10, grouped
gc.collect()

del df_train
gc.collect()

In [ ]:
## get test data, use it to filter the train data, and merge. 
df_train = pd.read_csv(file, usecols = ['custid','eventid','invest'])#,nrows=1000000)

In [ ]:
## custid bin can be done with this limited sample

## groupby custid
grouped1 = df_train[(df_train['invest']>=1.5) & (df_train['invest'] < 2.0)].\
groupby(['custid'])
grouped2 = df_train[(df_train['invest']>=2.0) & (df_train['invest'] < 10.0)].\
groupby(['custid'])
grouped10 = df_train[(df_train['invest']>=10.0)].groupby(['custid'])
grouped = df_train.groupby(['custid'])
bin_custid = pd.DataFrame({'meanInv.custid': grouped['invest'].mean().round(decimals=2), 
'rInvGT1.custid': (grouped1['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT2.custid': (grouped2['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT10.custid':(grouped10['invest'].count()/grouped['invest'].count()).round(decimals=4),
'q25Inv.custid':  grouped['invest'].quantile(0.25).round(decimals=2), 
'q50Inv.custid':  grouped['invest'].quantile(0.50).round(decimals=2), 
'q75Inv.custid':  grouped['invest'].quantile(0.75).round(decimals=2), 
'q95Inv.custid':  grouped['invest'].quantile(0.95).round(decimals=2), 
'q999Inv.custid':  grouped['invest'].quantile(0.999).round(decimals=2)})
del grouped2, grouped10, grouped
gc.collect()

In [ ]:
df_train['day'] = (np.rint(timestamp/(3600*24.)))

grouped1 = df_train[(df_train['invest']>=1.5) & (df_train['invest'] < 2.0)].\
groupby(['eventid'])
grouped2 = df_train[(df_train['invest']>=2.0) & (df_train['invest'] < 10.0)].\
groupby(['eventid'])
grouped10 = df_train[df_train['invest']>=10.0].groupby(['eventid'])
grouped = df_train.groupby(['eventid'])
bin_eventid = pd.DataFrame({'cnt.eventid': grouped['invest'].count(), 
#'daycnt.eventid': len(np.unique(grouped['day'].values)),
'rInvGT1.eventid': (grouped1['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT2.eventid': (grouped2['invest'].count()/grouped['invest'].count()).round(decimals=3),
'rInvGT10.eventid':(grouped10['invest'].count()/grouped['invest'].count()).round(decimals=4),
'q25Inv.eventid':  grouped['invest'].quantile(0.25).round(decimals=2), 
'q50Inv.eventid':  grouped['invest'].quantile(0.50).round(decimals=2), 
'q75Inv.eventid':  grouped['invest'].quantile(0.75).round(decimals=2), 
'q95Inv.eventid':  grouped['invest'].quantile(0.95).round(decimals=2)})
del grouped2, grouped10, grouped
gc.collect()

del df_train
gc.collect()

In [ ]:
## write out test result
df_test = pd.read_csv(file_te, usecols = ['custid','currency',\
                                          'eventid','platform','groupid','timestamp'])
df_test['index'] = np.arange(df_test.shape[0])

timestamp = df_test['timestamp'] - mintimestamp
df_test['time_hr'] = (np.rint(np.remainder(timestamp, 3600*24)/(divide)))

df_test = df_test.merge(bin_custid,\
                        left_on='custid',how='left',right_index=True)
df_test = df_test.merge(bin_eventid,\
                        left_on='eventid',how='left',right_index=True)
df_test = df_test.merge(bin_hr_currency,\
                        left_on=['custid','time_hr'], how='left',right_index=True)
df_test = df_test.merge(bin_hr_pltf,\
                        left_on=['platform','time_hr'], how='left',right_index=True)
df_test = df_test.merge(bin_hr_grpid,\
                        left_on=['groupid','time_hr'], how='left',right_index=True)

df_test.sort_index(inplace=True)

df_test.drop(['currency','platform','custid',\
              'eventid', 'groupid','timestamp'],axis=1,inplace=True)
print(df_test.columns)
print(df_test.shape) 
df_test.to_hdf(path+outputhead+'test.h5','test',format='table')
print('df_test ouput done!')
del df_test
gc.collect()

In [ ]:
## write out train

## load data
chunksize = 40730866/5
cols = ['custid','platform','currency', 'groupid','eventid','timestamp','invest']
ind = 0
nrows = 0
print('daycut {0}--{1}'.format(daycut_Rtr[0],daycut_Rtr[1]))
for chunk in pd.read_csv(file, usecols = cols,chunksize = chunksize):
    chunk['index'] = np.arange(chunk.shape[0]) + nrows
    nrows = nrows + chunk.shape[0]
#    chunk = chunk[chunk['custid'].isin(cust)]

    timestamp = chunk['timestamp'] - mintimestamp
    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], unit='s').map(lambda x: x.date())

    print('day range: ({0}, {1})'.format(chunk['timestamp'].min(), chunk['timestamp'].max()))
    if ( (chunk['timestamp'].min() > daycut_Rtr[1]) | (chunk['timestamp'].max() < daycut_Rtr[0])): 
        print('chunk {0} ignored!'.format(ind+1))
        ind = ind+1
        continue

    chunk = (chunk[ (chunk['timestamp'] <= daycut_Rtr[1]) & (chunk['timestamp'] > daycut_Rtr[0])] )
    chunk['time_hr'] = (np.rint(np.remainder(timestamp, 3600*24)/(divide)))
    
    chunk = chunk.merge(bin_custid, left_on='custid',how='left',right_index=True)
    chunk = chunk.merge(bin_eventid, left_on='eventid',how='left',right_index=True)
    chunk = chunk.merge(bin_hr_currency, left_on=['currency','time_hr'], how='left',right_index=True)
    chunk = chunk.merge(bin_hr_pltf, left_on=['platform','time_hr'], how='left',right_index=True)
    chunk = chunk.merge(bin_hr_grpid, left_on=['groupid','time_hr'], how='left',right_index=True)

    chunk.set_index('index',inplace=True)
    chunk.sort_index(inplace=True)
    chunk.drop(['groupid','currency','custid','eventid','platform'],axis=1, inplace=True)

    if(chunk['timestamp'].min() < daycut_Rtr[1]): 
        chunk_tr = chunk[(chunk['timestamp'] <= daycut_Rtr[1]) & (chunk['timestamp'] > daycut_Rtr[0])]
        chunk_tr.drop('timestamp',axis=1,inplace=True)
        cols = chunk.columns.tolist()
        ncols = cols[1:]
        ncols.append(cols[0])
        chunk = chunk[ncols]
        chunk_tr.to_hdf(path+outputhead+'train.h5','train', format='table', append=True)
        print('    --- to train.h5')

    cc = chunk.columns
    del chunk
    gc.collect()

    print('chunk {0} done!'.format(ind+1))
    ind = ind + 1

print('df_train output done!')
print('{0} rows is written in total.'.format(nrows))